In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.stattools import coint
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

coin list: BTC, ETH, SOL, UNI, APE, MANA

In [14]:
def get_resampled_close(path: str,resample: str ="1h"):
    df = pd.read_csv(path)
    df['open_time'] = df['open_time'].apply(lambda x: 
                                    datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    df.set_index('open_time',inplace=True)
    df = df[['close']]
    df = df.resample(resample).last()
    df.ffill(inplace=True)
    return df

In [15]:
btc = get_resampled_close("../data/BTCUSDT.csv")
eth = get_resampled_close("../data/ETHUSDT.csv")
# sol = get_resampled_close("../data/SOLUSDT.csv")
# uni = get_resampled_close("../data/UNIUSDT.csv")
# ape = get_resampled_close("../data/APEUSDT.csv")
# mana = get_resampled_close("../data/MANAUSDT.csv")

In [16]:
coint(btc,eth)

(-1.783049169943206,
 0.6379186033844976,
 array([-3.89694043, -3.33640917, -3.04464377]))

In [19]:
# Regress BTC on ETH
eth = sm.add_constant(eth)
result = sm.OLS(btc,eth).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  close   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.814
Method:                 Least Squares   F-statistic:                 9.600e+04
Date:                Sat, 30 Jul 2022   Prob (F-statistic):               0.00
Time:                        12:37:33   Log-Likelihood:            -2.2730e+05
No. Observations:               21889   AIC:                         4.546e+05
Df Residuals:                   21887   BIC:                         4.546e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.007e+04     85.109    118.263      0.0

In [21]:
# Compute ADF
b = result.params[1]
spread = btc['close'] - b * eth['close']
adf_stats = adfuller(spread)#,maxlag=1)
print("The p-value for the ADF test is ", adf_stats[1])
adf_stats

The p-value for the ADF test is  0.38889410523518664


(-1.783079324765677,
 0.38889410523518664,
 47,
 21841,
 {'1%': -3.4306494399853973,
  '5%': -2.861672342563964,
  '10%': -2.56684044222386},
 287425.4742031533)